In [23]:
%reset -f
import pandas as pd
import requests
import json

In [24]:
# API endpoints
url = 'https://fantasy.premierleague.com/api/bootstrap-static/'

In [25]:
response = requests.get(url)

if response.status_code == 200:
    print('The API request was successful.')
else:
    print(f'Error: The API request failed with status code {response.status_code}.')

The API request was successful.


In [26]:
pd.options.display.max_columns = 50
pd.options.display.max_rows = 100

In [27]:
response = requests.get(url)
players = response.json()['elements']

In [28]:
element_types = response.json()['element_types']
position_mapping = {}

for pos in element_types:
    position_mapping[pos['id']] = pos['singular_name']

In [29]:
player_info = []

for player in players:
    player_id = player['id']
    name = player['first_name'] + ' ' + player['second_name']
    position = position_mapping[player['element_type']]
    points = player['total_points']
    bonus = player['bonus']
    value = player['now_cost']/10
    minutes = player['minutes']
    ppg = player['points_per_game']  
    goals = player['goals_scored']
    xg = player['expected_goals']
    xg90 = player['expected_goals_per_90']
    assists = player['assists']
    xa = player['expected_assists']
    xa90 = player['expected_assists_per_90']
    cs = player['clean_sheets']
    goals_conc = player['goals_conceded']
    xgc = player['expected_goals_conceded']
    xgc90 = player['expected_goals_conceded_per_90']
    saves = player['saves']
    influence = player['influence']
    threat = player['threat']  
    creativity = player['creativity']
    transfers_in = player['transfers_in_event']
    transfers_out = player['transfers_out_event']
  
    player_info.append({
        'name': name,
        'pos': position[:1],
        'val': value,
        'mins': minutes,
        'pts': points,
        'pts-bns': int(points) - bonus,
        'ppg': ppg,
        'g': goals,
        'xg': float(xg),
        'xg90': xg90,
        'ass': assists,
        'xa': float(xa),
        'xa90': xa90,
        'cs': cs,
        'gc': goals_conc,
        'xgc': float(xgc),
        'xgc90': xgc90,
        'saves': saves,
        'influence': influence,
        'threat': threat,
        'creativity': creativity,
        'tra_in': transfers_in,
        'tra_out': transfers_out
    })
    
df = pd.DataFrame(player_info)

In [30]:
# Midfielders
mid = df.drop(columns=['cs', 'gc','saves','xgc','xgc90'])
mid.insert(6, 'xp', mid.assign(xp = mid['xa'] * 3 + mid['xg'] * 5)['xp'])
filtered_mid = mid[(mid['pos'] == 'M') & ((mid['xg90'] > 0.2) | (mid['xa90'] > 0.2)) & (mid['mins'] > 900) ]
sorted_mid = filtered_mid.sort_values(by='ppg', ascending=False)

styled_mid = sorted_mid.style.background_gradient(cmap='RdYlGn', subset=['g','ass','xa', 'xa90','xg', 'xg90']).format(precision=2)

styled_mid

,name,pos,val,mins,pts,pts-bns,xp,ppg,g,xg,xg90,ass,xa,xa90,influence,threat,creativity,tra_in,tra_out
391,Mohamed Salah,M,14.50,3374,344,289,150.47,9.1,29,24.70,0.66,18,8.99,0.24,1577.0,1985.0,1199.2,0,0
442,Bryan Mbeumo,M,8.00,3415,236,207,90.32,6.2,20,12.52,0.33,9,9.24,0.24,1236.8,1060.0,1107.5,0,0
239,Cole Palmer,M,10.50,3193,214,179,112.54,5.8,15,16.79,0.47,10,9.53,0.27,1068.2,1052.0,1259.2,0,0
462,Matheus Santos Carneiro da Cunha,M,8.00,2596,178,137,59.52,5.4,15,8.73,0.30,7,5.29,0.18,936.2,940.0,847.8,0,0
15,Bukayo Saka,M,10.00,1724,127,109,57.69,5.1,6,6.87,0.36,11,7.78,0.41,606.0,830.0,842.8,0,0
393,Luis Díaz Marulanda,M,8.00,2393,183,162,72.52,5.1,13,11.90,0.45,7,4.34,0.16,770.8,946.0,748.0,0,0
464,Amad Diallo,M,6.50,1898,129,112,35.49,5.0,8,4.71,0.22,8,3.98,0.19,634.0,615.0,687.2,0,0
461,Bruno Borges Fernandes,M,9.00,3017,174,146,73.32,4.8,8,9.93,0.30,12,7.89,0.24,1017.8,587.0,1407.7,0,0
123,Justin Kluivert,M,7.00,2339,158,144,61.23,4.6,12,10.05,0.39,6,3.66,0.14,700.0,771.0,654.1,0,0
425,Omar Marmoush,M,8.50,1174,73,61,32.18,4.6,7,5.41,0.41,1,1.71,0.13,318.0,410.0,246.7,0,0


In [31]:
# Forwards
forw = df.drop(columns=['cs', 'gc','saves','xgc','xgc90'])
forw.insert(6, 'xp', forw.assign(xp = forw['xa'] * 3 + forw['xg'] * 4)['xp'])
filtered_forw = forw[(forw['pos'] == 'F') & ((forw['xg90'] > 0.2) | (forw['xa90'] > 0.2)) & (forw['mins'] > 900) ]
sorted_forw = filtered_forw.sort_values(by='ppg', ascending=False)

styled_forw = sorted_forw.style.background_gradient(cmap='RdYlGn', subset=['g','ass','xa', 'xa90','xg', 'xg90']).format(precision=2)

styled_forw

,name,pos,val,mins,pts,pts-bns,xp,ppg,g,xg,xg90,ass,xa,xa90,influence,threat,creativity,tra_in,tra_out
510,Alexander Isak,F,10.50,2758,211,175,92.09,6.2,23,20.33,0.66,6,3.59,0.12,1043.0,1320.0,573.1,0,0
441,Erling Haaland,F,14.00,2736,181,155,93.72,5.8,22,21.90,0.72,3,2.04,0.07,946.0,1511.0,359.4,0,0
645,Jarrod Bowen,F,8.00,2974,193,172,53.81,5.7,13,8.60,0.26,11,6.47,0.20,903.0,1081.0,737.8,0,0
538,Chris Wood,F,7.50,2958,200,159,58.45,5.6,20,13.42,0.41,3,1.59,0.05,838.4,900.0,320.0,0,0
177,Yoane Wissa,F,7.50,2921,185,153,80.92,5.3,18,18.58,0.57,6,2.20,0.07,849.4,1181.0,400.0,0,0
65,Ollie Watkins,F,9.00,2593,186,154,66.69,4.9,16,15.12,0.52,8,2.07,0.07,766.0,1148.0,345.2,0,0
252,João Pedro Junqueira de Jesus,F,7.50,1946,126,100,45.14,4.7,10,8.96,0.41,6,3.10,0.14,567.0,644.0,456.0,0,0
29,Kai Havertz,F,7.50,1872,97,83,43.07,4.2,9,9.53,0.46,3,1.65,0.08,467.6,711.0,269.0,0,0
220,Danny Welbeck,F,6.50,2109,125,104,42.40,4.2,10,9.58,0.41,5,1.36,0.06,550.2,676.0,331.3,0,0
678,Jørgen Strand Larsen,F,6.50,2587,145,128,43.56,4.1,14,10.14,0.35,4,1.00,0.03,646.8,848.0,303.4,0,0


In [32]:
# Defenders
dfn = df.drop(columns=['saves'])
dfn.insert(6, 'xp', dfn.assign(xp = dfn['xa'] * 3 + dfn['xg'] * 6 + dfn['cs'] * 4 )['xp'])
filtered_dfn = dfn[(dfn['pos'] == 'D') & (dfn['mins'] > 900)]
sorted_dfn = filtered_dfn.sort_values(by='ppg', ascending=False)

styled_dfn = sorted_dfn.style.background_gradient(cmap='RdYlGn', subset=['g','ass','xa', 'xa90','xg', 'xg90', 'cs']).format(precision=2)
styled_dfn = styled_dfn.background_gradient(cmap='RdYlGn_r', subset=['gc', 'xgc','xgc90']).format(precision=2)

styled_dfn

,name,pos,val,mins,pts,pts-bns,xp,ppg,g,xg,xg90,ass,xa,xa90,cs,gc,xgc,xgc90,influence,threat,creativity,tra_in,tra_out
4,Gabriel dos Santos Magalhães,D,6.00,2363,117,108,59.83,4.2,3,2.70,0.10,2,1.21,0.05,10,22,23.09,0.88,584.6,287.0,208.8,0,0
415,Joško Gvardiol,D,6.00,3278,153,138,88.36,4.1,5,4.60,0.13,0,2.92,0.08,13,41,44.27,1.22,847.0,497.0,501.7,0,0
518,Nikola Milenković,D,5.50,3330,145,143,78.49,3.9,5,4.05,0.11,2,0.73,0.02,13,45,48.95,1.32,805.2,361.0,107.9,0,0
383,Virgil van Dijk,D,6.00,3330,143,135,72.26,3.9,3,2.22,0.06,1,0.98,0.03,14,38,36.13,0.98,932.6,299.0,206.6,0,0
485,Lewis Hall,D,5.50,2188,102,91,45.77,3.8,0,0.43,0.02,7,3.73,0.15,8,31,29.73,1.22,561.6,128.0,561.2,0,0
259,Daniel Muñoz Mejía,D,5.50,3229,142,133,83.90,3.8,4,4.92,0.14,6,3.46,0.10,11,46,46.30,1.29,816.6,524.0,634.2,0,0
7,Jurriën Timber,D,5.50,2415,114,105,55.28,3.8,1,1.23,0.05,3,1.30,0.05,11,21,21.09,0.79,393.0,254.0,296.8,0,0
520,Ola Aina,D,5.00,2996,128,113,58.53,3.7,2,0.63,0.02,1,2.25,0.07,12,39,40.59,1.22,578.2,80.0,282.3,0,0
230,Trevoh Chalobah,D,5.00,1969,93,82,37.80,3.7,3,1.98,0.09,1,0.64,0.03,6,25,27.15,1.24,496.4,222.0,124.2,0,0
228,Marc Cucurella Saseta,D,6.00,2988,133,119,59.91,3.7,5,2.79,0.08,2,2.39,0.07,9,38,39.94,1.20,668.6,358.0,350.6,0,0


In [33]:
# Goalies
gk = df.drop(columns=['xg','g','xg90','ass','xa','xa90','influence','threat','creativity'])
gk.insert(6, 'xp', gk.assign(xp = gk['cs'] * 4 + gk['xgc'] * -0.5 + gk['saves'] * 0.33 + gk['mins'] / 45)['xp'])
# gk.insert(7, 'diff', gk.assign(diff = gk['xp'] - gk['pts-bns'])['diff'])
filtered_gk = gk[(gk['pos'] == 'G') & (gk['mins'] > 180)]
sorted_gk = filtered_gk.sort_values(by='ppg', ascending=False)

styled_gk = sorted_gk.style.background_gradient(cmap='RdYlGn', subset=['cs', 'saves']).format(precision=2)
styled_gk = styled_gk.background_gradient(cmap='RdYlGn_r', subset=['gc', 'xgc','xgc90']).format(precision=2)

styled_gk

,name,pos,val,mins,pts,pts-bns,xp,ppg,cs,gc,xgc,xgc90,saves,tra_in,tra_out
144,Caoimhín Kelleher,G,4.50,900,45,44,39.37,4.5,4,12,11.09,1.11,27,0,0
105,Martin Dúbravka,G,4.00,900,44,42,43.98,4.4,5,12,11.19,1.12,29,0,0
411,Ederson Santana de Moraes,G,5.50,2320,111,102,94.63,4.3,10,26,29.50,1.14,54,0,0
291,Jordan Pickford,G,5.50,3420,158,143,141.21,4.2,12,44,46.11,1.21,122,0,0
376,Alisson Becker,G,5.50,2508,112,103,107.92,4.0,10,29,27.10,0.97,78,0,0
515,Matz Sels,G,5.00,3420,150,139,142.51,3.9,13,46,50.17,1.32,120,0,0
224,Robert Lynch Sánchez,G,5.00,2880,126,115,116.08,3.9,10,34,38.55,1.20,95,0,0
292,Mark Travers,G,4.50,450,19,18,16.92,3.8,1,5,7.36,1.47,20,0,0
0,David Raya Martín,G,5.50,3420,142,132,138.87,3.7,13,34,35.03,0.92,86,0,0
256,Dean Henderson,G,5.00,3420,135,127,129.16,3.6,11,51,50.33,1.32,104,0,0
